# Collaborative Filtering
Written by: Ryan Garnet Andrianto (Student ID: 05111940000063)

Learning source: https://spark.apache.org/docs/latest/ml-collaborative-filtering.html

## Download dataset
First, we download the dataset.

https://raw.githubusercontent.com/apache/spark/master/data/mllib/als/sample_movielens_ratings.txt

## Create spark session
After we install the pySpark python module, we can create a spark session and name it `collaborativeFiltering`.

In [3]:
from pyspark.sql import SparkSession

# Create spark session
spark = SparkSession \
    .builder \
    .appName('collaborativeFiltering') \
    .master('127.0.0.1[*]') \
    .getOrCreate()

spark   

## Import required library / module
In this collaborative filtering, we will need `RegressionEvaluator` to measure RMSE (Root-mean-square error), `ALS` (Alternating Least Squares), and `Row`. Thus, we import those class into our project.

In [4]:
from pyspark.ml.evaluation import RegressionEvaluator
from pyspark.ml.recommendation import ALS
from pyspark.sql import Row

## Read data and convert it to RDD (Resilient Distributed Datasets)
We read the dataset and store it as RDD.

In [5]:
lines = spark.read.text("datasets/sample_movielens_ratings.txt").rdd
parts = lines.map(lambda row: row.value.split("::"))
ratingsRDD = parts.map(lambda p: Row(userId=int(p[0]), movieId=int(p[1]),
                                     rating=float(p[2]), timestamp=int(p[3])))
ratings = spark.createDataFrame(ratingsRDD)
(training, test) = ratings.randomSplit([0.8, 0.2])

## Create the test cases
Our goal is to find a pair of maxIter and regParam value that has minimum RMSE value. In this case, we use maxIter = [5, 10, 20] and regParam = [0.01, 0.1, 0.5, 1.0]. The test case should be cross product of those unit tests.

In [6]:
# maxIter test
maxIter_test = [5, 10, 20]
regParam_test = [0.01, 0.1, 0.5, 1.0]

testCase = []
# Cross product between maxIter_test and regParam_test
for t1 in maxIter_test:
  for t2 in regParam_test:
    testCase.append({
        "maxIter": t1,
        "regParam": t2
    })

# Show test cases
print(testCase)

[{'maxIter': 5, 'regParam': 0.01}, {'maxIter': 5, 'regParam': 0.1}, {'maxIter': 5, 'regParam': 0.5}, {'maxIter': 5, 'regParam': 1.0}, {'maxIter': 10, 'regParam': 0.01}, {'maxIter': 10, 'regParam': 0.1}, {'maxIter': 10, 'regParam': 0.5}, {'maxIter': 10, 'regParam': 1.0}, {'maxIter': 20, 'regParam': 0.01}, {'maxIter': 20, 'regParam': 0.1}, {'maxIter': 20, 'regParam': 0.5}, {'maxIter': 20, 'regParam': 1.0}]


## Create the models
We create a model for each test cases.


In [7]:
# Create model for each test cases
models = []

for tc in testCase:
  # Build the recommendation model using ALS on the training data
  # Note we set cold start strategy to 'drop' to ensure we don't get NaN evaluation metrics
  als = ALS(maxIter=tc["maxIter"], regParam=tc["regParam"], userCol="userId", itemCol="movieId", ratingCol="rating",
            coldStartStrategy="drop")
  model = als.fit(training)

  models.append(model)

## Evaluate the models
After we create the models, we evaluate each of the models.



In [8]:
# Create regression evaluator
evaluator = RegressionEvaluator(metricName="rmse", labelCol="rating",
                                  predictionCol="prediction")
# Evaluate each model in the test cases
for id in range(0, len(testCase)):
  # Evaluate the model by computing the RMSE on the test data
  predictions = models[id].transform(test)
  rmse = evaluator.evaluate(predictions)
  print("For maxIter = " + str(testCase[id]["maxIter"]) + ", regParam = " + str(testCase[id]["regParam"]) + ", the Root-mean-square error = " + str(rmse))

For maxIter = 5, regParam = 0.01, the Root-mean-square error = 1.8013840045205773
For maxIter = 5, regParam = 0.1, the Root-mean-square error = 1.0093691081029197
For maxIter = 5, regParam = 0.5, the Root-mean-square error = 1.2292505037858257
For maxIter = 5, regParam = 1.0, the Root-mean-square error = 1.4967214174258427
For maxIter = 10, regParam = 0.01, the Root-mean-square error = 1.7691569698994294
For maxIter = 10, regParam = 0.1, the Root-mean-square error = 0.9689124386908716
For maxIter = 10, regParam = 0.5, the Root-mean-square error = 1.2275181548502707
For maxIter = 10, regParam = 1.0, the Root-mean-square error = 1.4967368556237997


Py4JJavaError: An error occurred while calling o843.evaluate.
: org.apache.spark.SparkException: Job aborted due to stage failure: Task 4 in stage 1261.0 failed 1 times, most recent failure: Lost task 4.0 in stage 1261.0 (TID 3518) (LAPTOP-1HPI5IML executor driver): java.lang.StackOverflowError
	at java.base/java.io.ObjectStreamClass.invokeReadObject(ObjectStreamClass.java:1175)
	at java.base/java.io.ObjectInputStream.readSerialData(ObjectInputStream.java:2379)
	at java.base/java.io.ObjectInputStream.readOrdinaryObject(ObjectInputStream.java:2250)
	at java.base/java.io.ObjectInputStream.readObject0(ObjectInputStream.java:1709)
	at java.base/java.io.ObjectInputStream.defaultReadFields(ObjectInputStream.java:2518)
	at java.base/java.io.ObjectInputStream.readSerialData(ObjectInputStream.java:2412)
	at java.base/java.io.ObjectInputStream.readOrdinaryObject(ObjectInputStream.java:2250)
	at java.base/java.io.ObjectInputStream.readObject0(ObjectInputStream.java:1709)
	at java.base/ja

----------------------------------------
Exception occurred during processing of request from ('127.0.0.1', 51856)
Traceback (most recent call last):
  File "c:\Users\Izuna\.conda\envs\spark-env\lib\socketserver.py", line 316, in _handle_request_noblock
    self.process_request(request, client_address)
  File "c:\Users\Izuna\.conda\envs\spark-env\lib\socketserver.py", line 347, in process_request
    self.finish_request(request, client_address)
  File "c:\Users\Izuna\.conda\envs\spark-env\lib\socketserver.py", line 360, in finish_request
    self.RequestHandlerClass(request, client_address, self)
  File "c:\Users\Izuna\.conda\envs\spark-env\lib\socketserver.py", line 747, in __init__
    self.handle()
  File "c:\Users\Izuna\.conda\envs\spark-env\lib\site-packages\pyspark\accumulators.py", line 262, in handle
    poll(accum_updates)
  File "c:\Users\Izuna\.conda\envs\spark-env\lib\site-packages\pyspark\accumulators.py", line 235, in poll
    if func():
  File "c:\Users\Izuna\.conda\envs

## Conclusion
By using maxIter = 10 and regParam = 0.1, the Root-mean-square error becomes 0.91 which is the lowest among the result.